## Import libraries and read in list of movie IDs

In [1]:
import imdb
import pandas
import time
import numpy as np

In [2]:
tmdb_data = pandas.read_csv("Data_with_pixels.csv")

In [3]:
movies_and_ids = tmdb_data[["imdb_id", "title"]]
movies_and_ids.columns = ['id','movie']

In [4]:
print(movies_and_ids.shape)
print(tmdb_data.shape)
movies_and_ids.head()

(36599, 2)
(36599, 16)


,id,movie
0,tt0168629,Dancer in the Dark
1,tt0314412,My Life Without Me
2,tt0120586,American History X
3,tt0315543,Open Hearts
4,tt0416320,Match Point


## Create IMDB object and test out connection

In [5]:
from imdb import IMDb

In [6]:
#ia = IMDb('http')
#print ia.get_keyword(u'story')

In [7]:
ia = imdb.IMDb()

In [8]:
s_result = ia.search_movie('The Princess Bride')

# Retrieves default information for the first result (a Movie object).
the_unt = s_result[0]
ia.update(the_unt)

# Print some information.
#print the_unt['runtimes']
print the_unt['rating']
director = the_unt['director'] # get a list of Person objects.


8.1


In [9]:
s2 = ia.get_movie(s_result[0].movieID)

In [10]:
sample_id = movies_and_ids['id'][0]
sample_id = sample_id[3:]
sample_movie = ia.get_movie(sample_id)
ia.update(sample_movie)
sample_movie.summary()

u'Movie\n=====\nTitle: Dancer in the Dark (2000)\nGenres: Crime, Drama, Musical.\nDirector: Lars von Trier.\nWriter: Lars von Trier.\nCast: Bj\xf6rk (Selma Jezkova), Catherine Deneuve (Kathy), David Morse (Bill Houston), Peter Stormare (Jeff), Joel Grey (Oldrich Novy).\nRuntime: 140.\nCountry: Denmark, Spain, Germany, Netherlands, Italy, USA, UK, France, Sweden, Finland, Iceland, Norway.\nLanguage: English, German, Czech.\nRating: 8.0 (83020 votes).\nPlot: 1964 in small town Washington state. Selma Jezkov\xe1, a Czechoslovakian immigrant, and her preteen son Gene live in a rented trailer owned by and on the property of married Bill and Linda Houston, he the town sheriff. Beyond Bill and Linda, Selma has a small group of friends who look out for her, including her primary confidante, Kathy, with who she works, and Jeff who wants to be her boyfriend. Jeff regularly waits outside Selma\'s workplace long before the end of her shift to drive her home, despite she always refusing in not want

In [47]:

for each in sample_movie['country']:
    print(each['name'])

François Truffaut


In [52]:
sample_movie['languages']

[u'French']

Features that are lists:
- director
- writer
- cast

## Function to extract data from IMDB

In [81]:
np.array(results)

array(['tt0168629',
       "[u'Argentina:16', u'Australia:MA15+', u'Brazil:12', u'Canada:14A', u'Denmark:15', u'Finland:K-14', u'France:Tous publics', u'Germany:12::(bw)', u'Hong Kong:IIB', u'Iceland:14::(original rating)', u'Iceland:16::(video rating)', u'Italy:T', u'Japan:R-15', u'Mexico:B', u'Netherlands:16', u'New Zealand:R13', u'Peru:14', u'Philippines:PG-13', u'Portugal:M/12', u'Singapore:PG', u'South Korea:12', u'Spain:13', u'Sweden:15', u'Switzerland:14::(canton of Geneva)', u'Switzerland:14::(canton of Vaud)', u'UK:15', u'USA:R']",
       'null',
       "[u'Argentina:16', u'Australia:MA15+', u'Brazil:12', u'Canada:14A', u'Denmark:15', u'Finland:K-14', u'France:Tous publics', u'Germany:12::(bw)', u'Hong Kong:IIB', u'Iceland:14::(original rating)', u'Iceland:16::(video rating)', u'Italy:T', u'Japan:R-15', u'Mexico:B', u'Netherlands:16', u'New Zealand:R13', u'Peru:14', u'Philippines:PG-13', u'Portugal:M/12', u'Singapore:PG', u'South Korea:12', u'Spain:13', u'Sweden:15', u'Switzer

In [73]:
alist = ['dog', 'cat', 'moo']

In [96]:
all_results.shape

(38,)

In [101]:
features_df['genres'][6]

"[u'Action', u'Comedy', u'Crime', u'Drama', u'Thriller']"

In [124]:
offset=0
interval=1
non_genre_string = np.array(['id','title', 'canonical title', 'imdbIndex','kind', 
                             'year', 'rating','mpaa','votes','runtimes',
                             'color info', 'genres','languages','plot','countries']) # 15 non-genre strings
non_genre_person_list = np.array(['director', 'writer', 'cast','certificates'])  # 4 person lists 

all_genres = np.array(['Action', 'Adventure', 'Adult', 'Animation',
'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir',
'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Short', 'Thriller',
'War', 'Western','Talk-Show', 'News', 'Game-Show', 'Reality-TV', 'History', 'Sport']) # 26 genres
all_results = np.concatenate((non_genre_string, non_genre_person_list, all_genres))


for num,each in enumerate(movies_and_ids['id'][offset:offset+interval]):
    #with Timer() as t:
    print(each)
    try:
        result = ia.get_movie(each[3:])
        print(result)
        if num%10==0: print(num)
        results = [each]
        for index, each in enumerate(non_genre_string[1:]):
            try:
                test = result[each]
                results.append(str(test))                
            except:
                results.append("null")

        for index, each in enumerate(non_genre_person_list):
            try:
                test = result[each]
                person_list = []
                for index,person in enumerate(test):
                    person_list.append(str(person['name']))
                results.append(str(person_list))
            except:
                results.append("null")

        for index, each in enumerate(all_genres):
            try:
                genre_result = result['genres']
                if each in genre_result:
                    results.append(1)

                else:
                    results.append(0)
            except:
                results.append("null")

    except:
        continue
    all_results = np.vstack((all_results, np.array(results)))

features_df = pandas.DataFrame(data=all_results[1:,],  columns=all_results[0,])
features_df

tt0168629
Help
0


,id,title,canonical title,imdbIndex,kind,year,rating,mpaa,votes,runtimes,...,Short,Thriller,War,Western,Talk-Show,News,Game-Show,Reality-TV,History,Sport
0,tt0168629,Help,Help,null,movie,null,null,null,null,null,...,null,null,null,null,null,null,null,null,null,null


In [15]:
import time
class Timer(object):
    def __init__(self, verbose=True):
        self.verbose = verbose

    def __enter__(self):
        self.start = time.time()
        return self

    def __exit__(self, *args):
        self.end = time.time()
        self.secs = self.end - self.start
        self.msecs = self.secs * 1000  # millisecs
        if self.verbose:
            print('elapsed time: %f ms %f projected finish' % (self.msecs, self.msecs*50000/(4*1000*3600)))

In [16]:
with Timer() as t:
    for i in range(1000):
        a = i*34

elapsed time: 0.885010 ms 0.003073 projected finish


In [17]:
def get_data(offset, interval, q):

    non_genre_string = np.array(['id','title', 'canonical title', 'imdbIndex','kind', 
                                 'year', 'rating','mpaa','votes','runtimes',
                                 'color info', 'genres','languages','plot','countries']) # 15 non-genre strings
    non_genre_person_list = np.array(['director', 'writer', 'cast','certificates'])  # 4 person lists 

    all_genres = np.array(['Action', 'Adventure', 'Adult', 'Animation',
    'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir',
    'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Short', 'Thriller',
    'War', 'Western','Talk-Show', 'News', 'Game-Show', 'Reality-TV', 'History', 'Sport']) # 26 genres
    all_results = np.concatenate((non_genre_string, non_genre_person_list, all_genres))


    for num,each in enumerate(movies_and_ids['id'][offset:offset+interval]):
        with Timer() as t:
            try:
                result = ia.get_movie(each[3:])
                if num%10==0: print(num)
                results = [each]
                for index, each in enumerate(non_genre_string[1:]):
                    try:
                        test = result[each]
                        results.append(str(test))                
                    except:
                        results.append("null")

                for index, each in enumerate(non_genre_person_list):
                    try:
                        test = result[each]
                        person_list = []
                        for index,person in enumerate(test):
                            person_list.append(str(person['name']))
                        results.append(str(person_list))
                    except:
                        results.append("null")

                for index, each in enumerate(all_genres):
                    try:
                        genre_result = result['genres']
                        if each in genre_result:
                            results.append(1)

                        else:
                            results.append(0)
                    except:
                        results.append("null")

            except:
                continue
            all_results = np.vstack((all_results, np.array(results)))

    features_df = pandas.DataFrame(data=all_results[1:,],  columns=all_results[0,])
    q.put(features_df)

## Use three processes to speed up the extraction

In [18]:
from multiprocessing import Process, Queue

elapsed_time = time.time()

if __name__ == '__main__':
    q = Queue()
    for offset in [0,100,200,300]:
        interval = 100
        Process(target=get_data, args=(offset,interval,q)).start()

0
elapsed time: 1622.985840 ms 5.635367 projected finish
0
elapsed time: 1812.191963 ms 6.292333 projected finish
0
0
elapsed time: 1913.864136 ms 6.645362 projected finish
elapsed time: 1925.252914 ms 6.684906 projected finish
elapsed time: 1250.603914 ms 4.342375 projected finish
elapsed time: 1681.985140 ms 5.840226 projected finish
elapsed time: 1980.309963 ms 6.876076 projected finish
elapsed time: 2265.404224 ms 7.865987 projected finish
elapsed time: 2174.201965 ms 7.549312 projected finish
elapsed time: 2746.865034 ms 9.537726 projected finish
elapsed time: 2228.425980 ms 7.737590 projected finish
elapsed time: 1081.448078 ms 3.755028 projected finish
elapsed time: 2730.321169 ms 9.480282 projected finish
elapsed time: 1635.582924 ms 5.679107 projected finish
elapsed time: 2131.037951 ms 7.399437 projected finish
elapsed time: 2131.464005 ms 7.400917 projected finish
elapsed time: 1469.521046 ms 5.102504 projected finish
elapsed time: 1228.344917 ms 4.265087 projected finish
el

2017-04-09 21:47:53,769 CRITICAL [imdbpy] //anaconda/envs/py27/lib/python2.7/site-packages/imdb/_exceptions.py:35: IMDbDataAccessError exception raised; args: ({'url': 'http://akas.imdb.com/title/tt0398027/combined', 'error type': 'http_error_default', 'errcode': 503, 'headers': <httplib.HTTPMessage instance at 0x11193a878>, 'proxy': '', 'errmsg': 'Service Temporarily Unavailable'},); kwds: {}
Traceback (most recent call last):
  File "<ipython-input-17-14a7d8a25897>", line 30, in get_data
    test = result[each]
  File "//anaconda/envs/py27/lib/python2.7/site-packages/imdb/utils.py", line 1472, in __getitem__
    rawData = self.data[key]
KeyError: 'certificates'
2017-04-09 21:47:53,822 CRITICAL [imdbpy] //anaconda/envs/py27/lib/python2.7/site-packages/imdb/__init__.py:777: caught an exception retrieving or parsing "main" info set for mopID "0398027" (accessSystem: http)
Traceback (most recent call last):
  File "//anaconda/envs/py27/lib/python2.7/site-packages/imdb/__init__.py", line 

elapsed time: 2989.610910 ms 10.380593 projected finish
elapsed time: 2329.176903 ms 8.087420 projected finish
50
elapsed time: 1282.649040 ms 4.453643 projected finish
elapsed time: 810.267210 ms 2.813428 projected finish


2017-04-09 21:47:54,435 CRITICAL [imdbpy] //anaconda/envs/py27/lib/python2.7/site-packages/imdb/_exceptions.py:35: IMDbDataAccessError exception raised; args: ({'url': 'http://akas.imdb.com/title/tt0061578/combined', 'error type': 'http_error_default', 'errcode': 503, 'headers': <httplib.HTTPMessage instance at 0x113d77440>, 'proxy': '', 'errmsg': 'Service Temporarily Unavailable'},); kwds: {}
Traceback (most recent call last):
  File "<ipython-input-17-14a7d8a25897>", line 33, in get_data
    person_list.append(str(person['name']))
TypeError: string indices must be integers
2017-04-09 21:47:54,458 CRITICAL [imdbpy] //anaconda/envs/py27/lib/python2.7/site-packages/imdb/__init__.py:777: caught an exception retrieving or parsing "main" info set for mopID "0061578" (accessSystem: http)
Traceback (most recent call last):
  File "//anaconda/envs/py27/lib/python2.7/site-packages/imdb/__init__.py", line 772, in update
    ret = method(mopID)
  File "//anaconda/envs/py27/lib/python2.7/site-pac

elapsed time: 580.387831 ms 2.015236 projected finish
elapsed time: 1639.755964 ms 5.693597 projected finish
elapsed time: 1694.501877 ms 5.883687 projected finish
elapsed time: 2995.368004 ms 10.400583 projected finish
elapsed time: 2626.034021 ms 9.118174 projected finish
elapsed time: 2568.455935 ms 8.918250 projected finish
elapsed time: 2537.147999 ms 8.809542 projected finish


...

80
elapsed time: 483.163118 ms 1.677650 projected finish
elapsed time: 3082.412004 ms 10.702819 projected finish
elapsed time: 1955.931902 ms 6.791430 projected finish
elapsed time: 1575.876951 ms 5.471795 projected finish
70
elapsed time: 2298.990965 ms 7.982608 projected finish
elapsed time: 2122.324944 ms 7.369184 projected finish
elapsed time: 1570.852041 ms 5.454347 projected finish
elapsed time: 2049.806118 ms 7.117382 projected finish
70
elapsed time: 2494.401932 ms 8.661118 projected finish
elapsed time: 2228.201866 ms 7.736812 projected finish
elapsed time: 3964.759111 ms 13.766525 projected finish
elapsed time: 2862.706900 ms 9.939955 projected finish
elapsed time: 3096.282959 ms 10.750982 projected finish
elapsed time: 1827.696085 ms 6.346167 projected finish
elapsed time: 2759.120941 ms 9.580281 projected finish
elapsed time: 2496.772051 ms 8.669347 projected finish
elapsed time: 1222.653866 ms 4.245326 projected finish
elapsed time: 1943.985939 ms 6.749951 projected finish

2017-04-09 21:48:53,204 CRITICAL [imdbpy] //anaconda/envs/py27/lib/python2.7/site-packages/imdb/_exceptions.py:35: IMDbDataAccessError exception raised; args: ({'url': 'http://akas.imdb.com/title/tt0043274/combined', 'error type': 'http_error_default', 'errcode': 503, 'headers': <httplib.HTTPMessage instance at 0x116759170>, 'proxy': '', 'errmsg': 'Service Temporarily Unavailable'},); kwds: {}
Traceback (most recent call last):
  File "<ipython-input-17-14a7d8a25897>", line 33, in get_data
    person_list.append(str(person['name']))
TypeError: string indices must be integers
2017-04-09 21:48:53,220 CRITICAL [imdbpy] //anaconda/envs/py27/lib/python2.7/site-packages/imdb/__init__.py:777: caught an exception retrieving or parsing "main" info set for mopID "0043274" (accessSystem: http)
Traceback (most recent call last):
  File "//anaconda/envs/py27/lib/python2.7/site-packages/imdb/__init__.py", line 772, in update
    ret = method(mopID)
  File "//anaconda/envs/py27/lib/python2.7/site-pac

elapsed time: 1738.292933 ms 6.035739 projected finish
elapsed time: 1905.189991 ms 6.615243 projected finish
elapsed time: 1951.267958 ms 6.775236 projected finish
elapsed time: 3436.002016 ms 11.930563 projected finish
elapsed time: 3087.522984 ms 10.720566 projected finish
elapsed time: 1896.018982 ms 6.583399 projected finish
elapsed time: 1661.334991 ms 5.768524 projected finish
elapsed time: 1954.524994 ms 6.786545 projected finish
elapsed time: 2218.539000 ms 7.703260 projected finish
elapsed time: 1469.388962 ms 5.102045 projected finish
elapsed time: 3693.293810 ms 12.823937 projected finish
80
elapsed time: 1089.254856 ms 3.782135 projected finish
elapsed time: 2783.833981 ms 9.666090 projected finish
elapsed time: 3150.156975 ms 10.938045 projected finish
elapsed time: 2106.336832 ms 7.313670 projected finish
elapsed time: 2561.324120 ms 8.893487 projected finish


2017-04-09 21:49:02,614 CRITICAL [imdbpy] //anaconda/envs/py27/lib/python2.7/site-packages/imdb/_exceptions.py:35: IMDbDataAccessError exception raised; args: ({'url': 'http://akas.imdb.com/title/tt0057007/combined', 'error type': 'http_error_default', 'errcode': 503, 'headers': <httplib.HTTPMessage instance at 0x111910ea8>, 'proxy': '', 'errmsg': 'Service Temporarily Unavailable'},); kwds: {}
Traceback (most recent call last):
  File "<ipython-input-17-14a7d8a25897>", line 33, in get_data
    person_list.append(str(person['name']))
TypeError: string indices must be integers
2017-04-09 21:49:02,647 CRITICAL [imdbpy] //anaconda/envs/py27/lib/python2.7/site-packages/imdb/__init__.py:777: caught an exception retrieving or parsing "main" info set for mopID "0057007" (accessSystem: http)
Traceback (most recent call last):
  File "//anaconda/envs/py27/lib/python2.7/site-packages/imdb/__init__.py", line 772, in update
    ret = method(mopID)
  File "//anaconda/envs/py27/lib/python2.7/site-pac

elapsed time: 778.123856 ms 2.701819 projected finish
80
elapsed time: 3689.100981 ms 12.809378 projected finish
elapsed time: 1754.568100 ms 6.092250 projected finish
elapsed time: 3020.623922 ms 10.488278 projected finish
elapsed time: 2162.328959 ms 7.508087 projected finish
elapsed time: 1752.999067 ms 6.086802 projected finish
elapsed time: 2088.174820 ms 7.250607 projected finish
elapsed time: 1474.586010 ms 5.120090 projected finish
elapsed time: 3569.133997 ms 12.392826 projected finish
elapsed time: 1126.529932 ms 3.911562 projected finish
elapsed time: 1496.069908 ms 5.194687 projected finish
elapsed time: 2381.459951 ms 8.268958 projected finish
elapsed time: 2490.663052 ms 8.648136 projected finish
elapsed time: 2063.633919 ms 7.165396 projected finish
elapsed time: 3165.166855 ms 10.990163 projected finish
elapsed time: 3304.306030 ms 11.473285 projected finish
elapsed time: 2843.882799 ms 9.874593 projected finish
elapsed time: 1968.949080 ms 6.836629 projected finish
ela

## Collect dataframes from the q and combine

In [19]:
all_dfs = []
for _ in range(4):
    all_dfs.append(q.get())

In [20]:
output_result = all_dfs[0]
for each in all_dfs[1:]:
    output_result = output_result.append(each,ignore_index=True)

## Check data and export to csv

In [22]:
output_result

,id,title,canonical title,imdbIndex,kind,year,rating,mpaa,votes,runtimes,...,Short,Thriller,War,Western,Talk-Show,News,Game-Show,Reality-TV,History,Sport
0,tt0942903,Stargate: The Ark of Truth,Stargate: The Ark of Truth,null,video movie,2008,7.4,null,16788,[u'97'],...,0,0,0,0,0,0,0,0,0,0
1,tt1488167,Navaho,Navaho,null,movie,1907,null,null,null,null,...,1,0,0,0,0,0,0,0,0,0
2,tt0799915,Air Guitar Nation,Air Guitar Nation,null,movie,2006,7.2,Rated R for some language and brief nudity,1231,[u'USA:81'],...,0,0,0,0,0,0,0,0,0,0
3,tt1182331,La mujer de medianoche,"mujer de medianoche, La",null,movie,1925,null,null,null,null,...,0,0,0,0,0,0,0,0,0,0
4,tt1176917,Laserdisc: How It Works,Laserdisc: How It Works,null,video movie,1980,null,null,null,[u'11'],...,1,0,0,0,0,0,0,0,0,0
5,tt0862846,Sunshine Cleaning,Sunshine Cleaning,null,movie,2008,6.9,"Rated R for language, disturbing images, some ...",62735,[u'91'],...,0,0,0,0,0,0,0,0,0,0
6,tt0369994,Strangers with Candy,Strangers with Candy,null,movie,2005,6.1,"Rated R for sexual content, language and some ...",7105,[u'97'],...,0,0,0,0,0,0,0,0,0,0
7,tt0828123,UFC 61: Bitter Rivals,UFC 61: Bitter Rivals,null,tv movie,2006,6.6,null,77,[u'USA:180'],...,0,0,0,0,0,0,0,0,0,1
8,tt0791304,Georgia Rule,Georgia Rule,null,movie,2007,5.9,Rated R for sexual content and some language,21065,[u'USA:113'],...,0,0,0,0,0,0,0,0,0,0
9,tt0103671,American Me,American Me,null,movie,1992,7.2,"Rated R for strong violence and sensuality, an...",6179,[u'125'],...,0,0,0,0,0,0,0,0,0,0


In [23]:
output_result.to_csv("imdb_full_dataset.csv")

### possible keys we can get from IMDB
title; string; the "usual" title of the movie, like "The Untouchables".
long imdb title; string; "Uncommon Valor (1983/II) (TV)"
canonical title; string; the title in the canonical format,
                         like "Untouchables, The".
long imdb canonical title; string; "Patriot, The (2000)".
year; string; the year of release or '????' if unknown.
kind; string; one in ('movie', 'tv series', 'tv mini series', 'video game',
                      'video movie', 'tv movie', 'episode')
imdbIndex; string; the roman number for movies with the same title/year.
director; Person list; a list of director's name (e.g.: ['Brian De Palma'])
cast; Person list; list of actor/actress, with the currentRole instance
                   variable set to a Character object which describe his
                   role/duty.
cover url; string; the link to the image of the poster.
writer; Person list; list of writers ['Oscar Fraley (novel)']
plot; list; list of plots and authors of the plot.
rating; string; user rating on IMDb from 1 to 10 (e.g. '7.8')
votes; string; number of votes (e.g. '24,101')
runtimes; string list; in minutes ['119'] or something like ['USA:118',
          'UK:116']
number of episodes; int; number or episodes for a series.
color info; string list; ["Color (Technicolor)"]
countries; string list; production's country ['USA', 'Italy']
genres; string list; one or more in (Action, Adventure, Adult, Animation,
		Comedy, Crime, Documentary, Drama, Family, Fantasy, Film-Noir,
		Horror, Musical, Mystery, Romance, Sci-Fi, Short, Thriller,
		War, Western) and other genres defined by IMDb.
akas; string list; list of aka for this movie
languages; string list; list of languages
certificates; string list; ['UK:15', 'USA:R']
mpaa; string; the mpaa rating
episodes (series only); dictionary of dictionary; one key for every season,
                        one key for every episode in the season.
number of episodes (series only); int; total number of episodes.
number of seasons (series only); int; total number of seasons.
series years (series only); string; range of years when the series was produced.
episode of (episode only); Movie object; the parent series for an episode.
season (episode only); int; the season number.
episode (episode only); int; the number of the episode in the season.
long imdb episode title (episode only); string; episode and series title.
series title; string.
canonical series title; string.